Author: Emilio Codecido

This code uses an API from NREL to download typical meteorological year (TMY) solar data (and other such as temp, wind) from the Indian satellite SUNY.

A model is fitted to data from 2000-2014, and the closest actual monthly data to the model is given through this TYM model.

The API requires point locations for queries, so a shape file outlining the districs in India is used to calculate the centroid lattidue and longitude of each district.

Data is provided in 60 mins intervals.

More information on the API can be found here. 
https://developer.nrel.gov/docs/solar/nsrdb/suny-india-tmy-data-download/

**NOTE**: You will need to get an API to download data without getting blocked.

In [1]:
import urllib.parse
import urllib.request
import json
import time
import matplotlib.pyplot as plt
import csv
import codecs
import sys, os
import numpy as np
import requests
import pandas as pd
import geopandas as gpd
import sys, os

import json

from IPython.display import display

from pandas import json_normalize

In [2]:
#Loading shape file for the districs
#Shape file obtained from: https://www.kaggle.com/datasets/imdevskp/india-district-wise-shape-files

path_to_data = '../Raw_data/India_districts2020.shp'
gdf = gpd.read_file(path_to_data)

#distname = gdf.distname[0]

#lat = gdf.geometry[0].centroid.y
#lon = gdf.geometry[0].centroid.x
gdf

,objectid,statecode,statename,state_ut,distcode,distname,distarea,totalpopul,totalhh,totpopmale,totpopfema,st_areasha,st_lengths,geometry
0,1,05,Uttarakhand,STATE,066,Nainital,4251.0,954605.0,191383.0,493666.0,460939.0,5.322546e+09,5.061827e+05,"POLYGON ((79.52659 29.05543, 79.52550 29.05545..."
1,2,05,Uttarakhand,STATE,060,Dehradun,3088.0,1696694.0,347001.0,892199.0,804495.0,4.177236e+09,5.781887e+05,"POLYGON ((77.87557 30.26052, 77.87467 30.26087..."
2,3,05,Uttarakhand,STATE,064,Almora,3144.0,622506.0,140577.0,291081.0,331425.0,4.140751e+09,4.634542e+05,"POLYGON ((79.28494 29.92735, 79.28495 29.92723..."
3,4,05,Uttarakhand,STATE,065,Champawat,1766.0,259648.0,53953.0,131125.0,128523.0,2.294297e+09,3.145080e+05,"POLYGON ((80.12479 29.01308, 80.12481 29.01306..."
4,5,05,Uttarakhand,STATE,056,Uttarkashi,8016.0,330086.0,67602.0,168597.0,161489.0,1.085166e+10,7.864256e+05,"POLYGON ((78.92267 31.25333, 78.93106 31.26840..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688,691,36,Telangana,STATE,T28,Wanaparthy,NaN,577758.0,123544.0,294833.0,282925.0,2.471575e+09,2.894403e+05,"POLYGON ((78.23115 16.43789, 78.23471 16.38793..."
689,692,36,Telangana,STATE,T27,Vikarabad,NaN,927140.0,194956.0,463350.0,463790.0,3.978808e+09,3.723430e+05,"POLYGON ((77.41953 17.12279, 77.41852 17.12334..."
690,693,36,Telangana,STATE,T13,Mahabubnagar,NaN,1486777.0,300508.0,745101.0,741676.0,5.627610e+09,4.420680e+05,"POLYGON ((77.49606 16.38155, 77.49382 16.38188..."
691,41,01,Ladakh,STATE,004,Kargil,14036.0,140802.0,18338.0,77785.0,63017.0,2.152617e+10,1.151337e+06,"POLYGON ((76.13869 34.63398, 76.13957 34.63408..."


In [11]:
#Using API to extract typical meteorological year data for each district.

api_key = ''
email = ''

attributes = 'ghi'
#attributes = 'dhi, dni, ghi, clearsky_dhi, clearsky_dni, clearsky_ghi, dew_point, surface_temperature, surface_pressure, relative_humidity, solar_zenith_angle, total_precipitable_water, snow_depth, wdir, wspd, fill_flag'

#Extracting data for each district
for i in range(0,693):
    
    #Extracting centroid points for each district
    dist = gdf.distname[i]
    lat = gdf.geometry[i].centroid.y
    lon = gdf.geometry[i].centroid.x

    wkt = 'POINT('+ str(lon) + '%20' + str(lat) + ')'


    # Declare url string
    url = 'https://developer.nrel.gov/api/nsrdb/v2/solar/suny-india-tmy-download.csv?'
    url = url + 'api_key={api}&wkt={wkt}&names=tmy&utc=false&email={email}'
    url = url.format(wkt=wkt, api=api_key, attributes=attributes, names=years, email=email)
    
    #Put data in Pandas
    data = pd.read_csv(url, skiprows=2)
    data["District"] = dist
    data["Longitude"] = lon
    data["Lattitude"] = lat
    data = data[['District','Longitude','Lattitude', 'Year','Month', 'Day', 'Hour', 'Minute', 'Dew Point', 'DHI', 'DNI', 'GHI', 'Pressure', 'Temperature', 'Wind Direction', 'Wind Speed',]]
    data = data.rename(columns={"Dew Point": "Dew Point (C)", "DHI": "DHI (W/m^2)", "DNI": "DNI (W/m^2)", "GHI": "GHI (W/m^2)", "Pressure": "Pressure (mbar)", "Temperature": "Temperature (C)", "Wind Direction": "Wind Direction (degrees)","Wind Speed": "Wind Speed (m/s)"})
    
    filename = dist + ".xlsx"
    
    #Save to excel file
    data.to_excel(filename)

In [12]:
#Example output of the data for a typical meteorological year (TMY) in Nainital.
data

,District,Longitude,Lattitude,Year,Month,Day,Hour,Minute,Dew Point (C),DHI (W/m^2),DNI (W/m^2),GHI (W/m^2),Pressure (mbar),Temperature (C),Wind Direction (degrees),Wind Speed (m/s)
0,Nainital,79.467373,29.311269,2002,1,1,0,30,-6,0,0,0,8980,7,4.8,0.1
1,Nainital,79.467373,29.311269,2002,1,1,1,30,-7,0,0,0,8980,7,5.2,0.1
2,Nainital,79.467373,29.311269,2002,1,1,2,30,-7,0,0,0,8980,7,5.4,0.1
3,Nainital,79.467373,29.311269,2002,1,1,3,30,-8,0,0,0,8980,7,6.0,0.0
4,Nainital,79.467373,29.311269,2002,1,1,4,30,-9,0,0,0,8990,6,6.6,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,Nainital,79.467373,29.311269,2001,12,31,19,30,-3,0,0,0,8980,8,1.9,0.1
8756,Nainital,79.467373,29.311269,2001,12,31,20,30,-3,0,0,0,8980,8,2.6,0.1
8757,Nainital,79.467373,29.311269,2001,12,31,21,30,-4,0,0,0,8980,7,2.9,0.1
8758,Nainital,79.467373,29.311269,2001,12,31,22,30,-4,0,0,0,8980,7,3.2,0.0
